# Bidding

This is a tutorial for how to use the bidding engine

In [1]:
import os
os.chdir('..')

from nn.models_tf2 import Models
from botbidder import BotBid
from util import hand_to_str
from sample import Sample
import conf
import numpy as np

np.set_printoptions(precision=2, suppress=True, linewidth=200)
np.random.seed(42)
models = Models.from_conf(conf.load('./config/default.conf'),'..')   # loading neural networks
sampler = Sample.from_conf(conf.load('./config/default.conf'), False)  # Load sampling strategies


Loading config file d:\GitHub\ben\src\./config/default.conf
Loading config file d:\GitHub\ben\src\./config/default.conf


### Running through an example

In [2]:
# East deals, EW vulnerable.
vuln_ns, vuln_ew = False, False

from ddsolver import ddsolver
dds = ddsolver.DDSolver()

# you sit West and hold:
hand = 'K62.Q7542.J832.7'

auction = ["PASS", "2N", "PASS", "3D","PASS", "4C", "PASS"]
bot_bid = BotBid([vuln_ns, vuln_ew], hand, models, sampler, 3, 0, dds, False, False)

bid = bot_bid.bid(auction)
bid.to_dict()['candidates']

# what's your bid?

[{'call': '4H',
  'insta_score': 0.673,
  'adjustment': 5.38,
  'expected_score': 404,
  'expected_imp': 0.67,
  'expected_tricks': 11.3},
 {'call': '4D',
  'insta_score': 0.36,
  'adjustment': 2.88,
  'expected_score': 404,
  'expected_imp': 0.67,
  'expected_tricks': 11.3,
  'alert': 'True',
  'explanation': 'Cue bid, surplus -- 5+!H; 6-!S; 5-20 HCP; Artificial; Forcing'},
 {'call': '4N',
  'insta_score': 0.095,
  'adjustment': -3.24,
  'expected_score': 347,
  'expected_imp': -1.33,
  'expected_tricks': 11.3}]

in the output above:
- `insta_score` reflects what the neural network prefers to bid
- `expected_score` is based on what you expect to score on difference samples (for example, 70% of the time you are making and 30% of the time you are going one off, then the expected score is 0.7 * 620 - 0.3 * 100)

### Samples consistent with the auction so far

Above we said that we are computig expected scores on samples consistent with the auction so far.

How do we get these samples?

We get some samples in the response we get from the bot (i.e `bid.samples`)

In [3]:
bid.samples

['Txxx.xxx.QTxx.Ax AQ.AKJT.Kx.KQxxx Jxxx.x.Axx.JTxxx Kxx.Qxxxx.Jxxx.x - 0.51493 | P-2N-P-3D-P-4C-P-4H-P-P-P (11) | P-2N-P-3D-P-4C-P-4D-P-4N-P-5C-P-5H-P-P-P (11) | P-2N-P-3D-P-4C-P-4N-P-5C-P-5H-P-P-P (11)',
 'Axxx.Txx.Kx.Jxxx Qxx.AKJx.A.AKTxx JTx.x.QTxxxx.Qxx Kxx.Qxxxx.Jxxx.x - 0.51077 | P-2N-P-3D-P-4C-P-4H-P-P-P (12) | P-2N-P-3D-P-4C-P-4D-P-4N-P-5C-P-5H-P-P-P (12) | P-2N-P-3D-P-4C-P-4N-P-5D-P-5H-P-P-P (12)',
 'Txx.Tx.ATxxx.xxx Ax.AKxx.Qx.AKQTx QJxxx.Jx.Kx.Jxxx Kxx.Qxxxx.Jxxx.x - 0.50749 | P-2N-P-3D-P-4C-P-4H-P-P-P (11) | P-2N-P-3D-P-4C-P-4D-P-4N-P-5C-P-5H-P-P-P (11) | P-2N-P-3D-P-4C-P-4N-P-5D-P-5H-P-P-P (11)',
 'Txxx.Tx.Txxxx.Kx AQx.AKxx.x.AQJxx Jxx.Jx.AKQ.Txxxx Kxx.Qxxxx.Jxxx.x - 0.50145 | P-2N-P-3D-P-4C-P-4H-P-P-P (12) | P-2N-P-3D-P-4C-P-4D-P-4N-P-5C-P-5H-P-P-P (12) | P-2N-P-3D-P-4C-P-4N-P-5D-P-5H-P-P-P (12)',
 'QJxxx.Tx.AKTx.xx Axx.AKJx.x.AKJTx Tx.xx.Qxxx.Qxxxx Kxx.Qxxxx.Jxxx.x - 0.45021 | P-2N-P-3D-P-4C-P-4H-P-P-P (12) | P-2N-P-3D-P-4C-P-4D-P-4N-P-5C-P-5H-P-P-P (12) | P-2N-P-3D-P-4

each row is one sample. the hands are in the order: N E S W

(West always has the same cards because it's your hand and it's known to you)

In [5]:
# we got some samples above. if we want more, we can always get more

sample_hands, sorted_score, p_hcp, p_shp, good_quality = bot_bid.sample_hands_for_auction(["PAD_START", "PAD_START", "PAD_START", "PASS", "1N", "PASS", "3S","PASS"],0)
print("Good quality:", good_quality)
for i in range(sample_hands.shape[0]):
    print(f'{hand_to_str(sample_hands[i, 0, :], models.n_cards_bidding)} {hand_to_str(sample_hands[i, 1, :], models.n_cards_bidding)} {hand_to_str(sample_hands[i, 2, :], models.n_cards_bidding)} {hand_to_str(sample_hands[i, 3, :], models.n_cards_bidding)}')

Good quality: 0.7977053319673403
Kxx.Qxxxx.Jxxx.x AQJx.AJx.xx.QTxx x.Kxxx.KQTx.AKxx Txxxx.T.Axx.Jxxx
Kxx.Qxxxx.Jxxx.x QTxxx.Jx.x.Axxxx x.ATxx.AKQx.KTxx AJxx.Kx.Txxx.QJx
Kxx.Qxxxx.Jxxx.x AJTxx.KJT.QTx.QJ x.Axxx.Axxx.KTxx Qxxx.x.Kx.Axxxxx
Kxx.Qxxxx.Jxxx.x QJTx.Jx.AQxx.KTx A.KTxx.KTxx.QJxx xxxxx.Ax.x.Axxxx
Kxx.Qxxxx.Jxxx.x QTxx.AKx.Tx.KJxx A.JTxx.AKxx.ATxx Jxxxx.x.Qxx.Qxxx
Kxx.Qxxxx.Jxxx.x AJTxxx.x.Qxx.Txx .JTxx.AKTx.AKQJx Qxxx.AKx.xx.xxxx
Kxx.Qxxxx.Jxxx.x AQJTxx.x.Txx.Jxx .KJxx.AKQx.KTxxx xxxx.ATx.xx.AQxx
Kxx.Qxxxx.Jxxx.x ATxxx.AKx.xx.Jxx x.Jxxx.AQTx.AKxx QJxx.T.Kxx.QTxxx
Kxx.Qxxxx.Jxxx.x AQxx.KTx.ATxx.Tx J.Jxxx.KQxx.AJxx Txxxx.A.x.KQxxxx
Kxx.Qxxxx.Jxxx.x Axxxx.Txx.KT.Axx x.AKJx.Axxx.Jxxx QJTx.x.Qxx.KQTxx
Kxx.Qxxxx.Jxxx.x AQxx.K.QTx.QJTxx .AJTx.AKxx.AKxxx JTxxxx.xxx.xx.xx
Kxx.Qxxxx.Jxxx.x QJxx.A.Txx.AJxxx A.KTxx.Axxx.QTxx Txxxx.Jxx.KQ.Kxx
Kxx.Qxxxx.Jxxx.x Qxxxx.x.Txx.AQJx T.AKxx.KQxx.Kxxx AJxx.JTx.Ax.Txxx
Kxx.Qxxxx.Jxxx.x AQTx.Kxx.xx.Kxxx x.ATxx.ATxx.AQxx Jxxxx.J.KQx.JTxx
Kxx.Qxxxx.Jxxx.